In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report
import numpy as np



/ihome/xli/sek188/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bias_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bias_model = AutoModelForSequenceClassification.from_pretrained("bucketresearch/politicalBiasBERT")

tqdm.pandas()

/ihome/crc/install/pytorch/2.0.1/python3.10/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/ihome/crc/install/pytorch/2.0.1/python3.10/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/ihome/crc/install/pytorch/2.0.1/python3.10/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out ht

In [17]:
#make dataset
wdf = pd.read_csv('combined_wide_form_jul30.csv')
wdf = wdf[['article_set_unique_id', 'source_url', 'title', 'text', 'human_clean_title', 'human_clean_text', 'pull']]
wdf['clean'] = 'HEADLINE: ' + wdf['human_clean_title']

# wdf['machine_full_clean'] = 'HEADLINE: ' + df['human_clean_title'] + '\n\n' + 'STORY TEXT: ' + df['human_clean_text']

In [20]:
def pol_bias_reward_func(text):

    inputs = bias_tokenizer(text, return_tensors="pt")
    labels = torch.tensor([0])
    outputs = bias_model(**inputs, labels=labels)
    loss, logits = outputs[:2]
    scores = logits.softmax(dim=-1)[0].tolist()
    return scores




In [21]:
# wdf = wdf.sample(2)
wdf['baly_scores'] = wdf['clean'].apply(pol_bias_reward_func)
wdf['baly_pred'] = wdf['baly_scores'].apply(lambda x: (x.index(max(x)))-1)

In [23]:
# wdf.reset_index(drop=True, inplace=True)
# wdf.loc[0, 'baly_scores']
# wdf.shape
wdf['baly_pred'].value_counts()

baly_pred
 0    428
-1      1
Name: count, dtype: int64

In [19]:
checker = wdf[wdf['clean'].isna()]
checker.head()

,article_set_unique_id,source_url,title,text,human_clean_title,human_clean_text,pull,clean


In [22]:
wdf.to_csv('baly_wide_form_jul30.csv')